In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import division, print_function, absolute_import
import glob
import pickle
import matplotlib.pyplot as plt
import numpy as np
import scipy as sc
import os
from scipy import stats
import sys
sys.path.append('..')
from edfreader import read_edf

In [2]:
%matplotlib inline
fig_width = 15

from aSPEM import aSPEM
import bayesianchangepoint as bcp

In [3]:
liste = []
for fname in glob.glob('data/*pkl'):
    a = fname.split('/')[1].split('.')[0].split('_')
    liste.append(a)
    #print(a)

exp = []
for x in range(len(liste)) :
    if liste[x][0]=='enregistrement' and liste[x][1] in ['AM','BMC','CS','DC','FM','IP','LB','OP','RS','SR','TN','YK']:
        a = 'data/%s_%s_%s_%s.pkl'%(liste[x][0], liste[x][1],liste[x][2],liste[x][3])
        with open(a, 'rb') as fichier :
            b = pickle.load(fichier, encoding='latin1')
            exp.append(b)

In [4]:
timeStr = {}
timeStr['AM'] = '2017-10-23_100057'
timeStr['BMC'] = '2017-09-26_095637'
timeStr['CS'] = '2017-10-03_134421'
timeStr['DC']= '2017-09-27_161040'
timeStr['FM'] = '2017-10-03_143803'
timeStr['IP'] = '2017-09-28_115250'
timeStr['LB'] = '2017-09-20_151043'
timeStr['OP'] = '2017-10-26_121823'
timeStr['RS'] = '2017-11-08_094717'
timeStr['SR'] = '2017-11-16_153313'
timeStr['TN'] = '2017-11-08_150410'
timeStr['YK'] = '2017-11-17_172706'

In [5]:
def Microsaccade (grad, VFAC=6, mindur=8, maxdur=100):
    '''
    mindur et maxdur en ms
    '''
    msdx = np.sqrt((np.nanmedian(grad**2))-((np.nanmedian(grad))**2))

    radiusx = VFAC*msdx

    test = (grad/radiusx)**2
    index = [x for x in range(len(test)) if test[x] > 1]
    
    dur = 0
    debut_sac = 0
    k = 0
    
    sac = []
    for k in np.arange(1, len(index)-1, 1) :
        if index[k+1]-index[k]==1 :
            dur = dur + 1;
        else :
            if dur >= mindur and dur < maxdur :
                fin_sac = k
                sac.append([index[debut_sac], index[fin_sac]])
            debut_sac = k+1
            dur = 1
        k = k + 1
    return sac

In [6]:
def suppression_saccades(data_x, trackertime, saccades, trackertime_0):
    gradient_x = np.gradient(data_x) # gradient en px/ms
    gradient_deg = gradient_x * 1/exp[0]['px_per_deg'] * 1000 # gradient en deg/sec
    
    gradient_deg_NAN = gradient_deg
    
    for s in range(len(saccades)) :
        if saccades[s][1]-trackertime_0+15 <= (len(trackertime)) :
            for x_data in np.arange((saccades[s][0]-trackertime_0-5), (saccades[s][1]-trackertime_0+15)) :
                gradient_deg_NAN[x_data] = np.nan
        else :
            for x_data in np.arange((saccades[s][0]-trackertime_0-5), (len(trackertime))) :
                gradient_deg_NAN[x_data] = np.nan
    return gradient_deg_NAN

In [7]:
def fig_velocity(ax, sujet, block, trial, t_titre=35, t_label=20) :

    trial_data = trial + 200*block
    
    resultats = os.path.join('..','data', 'enregistrement_' + sujet + '_' + timeStr[sujet] + '.asc')
    data = read_edf(resultats, 'TRIALID')
    
    data_x = data[trial_data]['x']
    trackertime = data[trial_data]['trackertime']

    StimulusOn = data[trial_data]['events']['msg'][10][0]
    StimulusOf = data[trial_data]['events']['msg'][14][0]
    TargetOn = data[trial_data]['events']['msg'][15][0]
    TargetOff = data[trial_data]['events']['msg'][16][0]
    saccades = data[trial_data]['events']['Esac']
    trackertime_0 = data[trial_data]['trackertime'][0]

    start = TargetOn
    StimulusOn_s = StimulusOn - start
    StimulusOf_s = StimulusOf - start
    TargetOn_s = TargetOn - start
    TargetOff_s = TargetOff - start
    trackertime_s = trackertime - start

    grad = suppression_saccades(data_x, trackertime, saccades, trackertime_0)
    sac = Microsaccade (grad)

    ax.axvspan(StimulusOn_s, StimulusOf_s, color='k', alpha=0.2)
    ax.axvspan(StimulusOf_s, TargetOn_s, color='r', alpha=0.2)
    ax.axvspan(TargetOn_s, TargetOff_s, color='k', alpha=0.15)

    debut  = TargetOn - trackertime_0
    grad2=np.copy(grad)

    for s in range(len(saccades)) :
        ax.axvspan(saccades[s][0]-start, saccades[s][1]-start, color='k', alpha=0.15)
    for s in range(len(sac)) :
        ax.axvspan(sac[s][0]-debut, sac[s][1]-debut, color='r', alpha=0.3)

        if sac[s][1]+15 <= (len(trackertime)) :
            for x_data in np.arange((sac[s][0]-5), (sac[s][1]+15)) :
                grad2[x_data] = np.nan
        else :
            for x_data in np.arange((sac[s][0]-5), (len(trackertime))) :
                grad2[x_data] = np.nan
                
    ax.plot(trackertime_s, grad, color='k', alpha=0.4)
    ax.plot(trackertime_s, grad2, color='r', alpha=0.4)

    ax.axis([-750, 750, -39.5, 39.5])
    ax.xaxis.set_ticks_position('bottom')
    ax.xaxis.set_tick_params(labelsize=t_label/2)
    ax.yaxis.set_ticks_position('left')
    ax.yaxis.set_tick_params(labelsize=t_label/2)
    ax.set_xlabel('Time (ms)', fontsize=t_label)

In [8]:
fig, ax = plt.subplots(50,1, figsize=(15, 50*15/1.6180))

for x in range(50) :
    fig_velocity(ax[x], sujet='CS', block=1, trial=x,  t_titre=35, t_label=20) 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
